In [ ]:
from_drive=True

In [ ]:
## GIT ON COLAB ONLY
try:
    from google.colab import userdata
    git_token = userdata.get('gitToken')
    git_user = userdata.get('gitUser')
    git_url = f'https://{git_token}@github.com/rene-aum/aum-dashboard.git'
    branch_to_pull = 'dev'
    !git clone {git_url}
    %cd aum-dashboard
    !git checkout -b colab_branch
    !git pull origin {branch_to_pull}
    !pip install -r dashboard_liderazgo/src/requirements.txt
    %cd dashboard_liderazgo
except Exception as e:
    print(e)
    print('Running in other environment not colab probably!')

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read)
from src.loader import Loader
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_file_in_drive_folder,
                             update_file_in_drive_folder,
                             read_from_google_sheets)
from src.constants import (
                           id_tracking_envios,
                           id_reporte_ventas,
                           id_business_case,
                           id_master_table_sheets,
                           id_clientes_apartado_sheets,
                           id_resultados_squad,
                           id_business_case_processed,
                           id_medios_total_sheets,
                           id_medios_propios_consolidado_sheets
                           )


warnings.filterwarnings('ignore')



In [ ]:
if from_drive:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import gspread
    from google.auth import default
    from gspread_dataframe import set_with_dataframe
    import gdown
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    ######################################################### get ids per file ###############################################################
    folder_id = '1hl7hITBmAXZKkb_qAHZ-045yJnSwfU2Q'

    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    file_id_dict = {}
    for file in file_list:
        file_id_dict[file['title']] = file['id']
    id_data_warehouse_drive = file_id_dict['Data Warehouse - Productivo.xlsx']
    id_CTA_adobe_csv = file_id_dict['CTAAdobe.csv']
    id_unique_visitors_adobe = file_id_dict['UniqueVisitorsAdobe.csv']
    id_medios_total_csv = file_id_dict['medios_total.csv']
    id_cancelaciones = file_id_dict['VS_cancellation_log.csv']
    id_medios_propios_consolidado_csv = file_id_dict['medios_propios_consolidado.csv']
    id_channel_appstep = file_id_dict['ChannelAppStep.csv']
    id_product_views = file_id_dict['pdp-hist.csv']
    ##################################################### get all files to local filesystem ###################################################
    from_drive_to_local(drive,id_data_warehouse_drive,'dw_automarket.xlsx')
    from_drive_to_local(drive,id_tracking_envios,'tracking_campañas_automarket_datio.xlsx')
    from_drive_to_local(drive,id_unique_visitors_adobe,'unique_visitors_adobe.csv')
    from_drive_to_local(drive,id_CTA_adobe_csv,'CTAAdobe.csv')
    from_drive_to_local(drive,id_medios_total_csv, 'medios_total.csv')    
    from_drive_to_local(drive,id_cancelaciones,'cancelaciones.csv')
    from_drive_to_local(drive,id_medios_propios_consolidado_csv, 'medios_propios_consolidado.csv')
    from_drive_to_local(drive,id_channel_appstep,'ChannelAppStep.csv')
    from_drive_to_local(drive,id_product_views,'pdp-hist.csv')
    ###########################################################################################################################################
    data_date = get_last_modification_date_drive(drive,id_data_warehouse_drive)
    ###########################################################################################################################################
    creds, _ = default()
    gc = gspread.authorize(creds)

In [ ]:
lo = Loader()

In [ ]:
path_excel = 'dw_automarket.xlsx'#'/Users/mi36597/Documents/automarket/dw_productivo_raw/dw_productivo_20250213.xlsx'
path_envios = 'tracking_campañas_automarket_datio.xlsx'
path_visitors_adobe = 'unique_visitors_adobe.csv'#"/Users/mi36597/Documents/automarket/UniqueVisitorsAdobe.csv"
path_salud_inventario = 'salud_de_inventario_completo.csv'
path_CTA_adobe = 'CTAAdobe.csv'
path_medios_total = 'medios_total.csv'
path_medios_propios_consolidado = 'medios_propios_consolidado.csv'
path_channel_appstep = 'ChannelAppStep.csv'
path_pdp = 'pdp-hist.csv'

# Load Data

In [ ]:
all_dates = get_dates_dataframe()

In [ ]:
try:
    envios_df = lo.load_envios_medios_propios(excel_path = path_envios, excel_tab_name = 'total')
except Exception as e:
    print(e)

In [ ]:
reporte_ventas = read_from_google_sheets(gc, id_reporte_ventas)
business_case_df = read_from_google_sheets(gc,id_business_case,sheetname='Looker')

In [ ]:
unique_visitors = lo.load_unique_visitors_adobe(csv_path=path_visitors_adobe)
pedidos_logs = lo.load_pedidos_tab(excel_path=path_excel,excel_tab_name='Pedidos')
new_users = lo.load_clientes_tab(excel_path=path_excel,excel_tab_name='Clientes')
veh_status = lo.load_vehicle_status_tab(excel_path=path_excel,excel_tab_name='Vehicle Status')
appstep_df = lo.load_appstep_tab(csv_path=path_channel_appstep)
apart = lo.load_pedidos_tab(excel_path=path_excel,excel_tab_name='Pedidos')
tb = lo.load_terminos_de_busqueda_tab(excel_path=path_excel,excel_tab_name='Términos Adobe')
p_views  = lo.load_product_views_tab(csv_path=path_pdp)
citas = lo.load_citas_tab(excel_path=path_excel,excel_tab_name='Citas')
CTA = lo.load_cta_adobe(csv_path = path_CTA_adobe)
cancelaciones = lo.load_cancelaciones(csv_path = 'cancelaciones.csv')

In [ ]:
medios_total = pd.read_csv(path_medios_total)
medios_propios_consolidado = pd.read_csv(path_medios_propios_consolidado)

# Transform data

In [ ]:
tr = Transformed(lo)

In [ ]:
new_users_data = tr.t_new_users_by_date()
incio_publicacion_df = tr.t_inicio_publicacion_by_date()
publicacion_df = tr.t_publicacion_by_date()
publicacion_historic = tr.t_publicacion_historica_by_date()
product_views_by_date = tr.t_product_views_by_date()
apartados_df = tr.t_apartados_historico_by_date()
terminos_busqueda_df = tr.t_terminos_de_busqueda()
apartados_activos = tr.t_apartados_activos_by_date()
inicio_apartado_df = tr.t_inicio_apartado_by_date()
reporte_ventas_mod = tr.t_ventas_by_date(reporte_ventas)
unique_visitors_mod = tr.t_unique_visitors_adobe_by_date()
apartados_transaccionables = tr.t_apartados_transaccionables()
citas_efectivas = tr.t_citas_efectivas_by_date()
ventas_garantia = tr.t_ventas_garantia_by_date(reporte_ventas)
seguros = tr.t_seguros_by_date(reporte_ventas)
registros_tipo_date = tr.t_registros_tipo_by_date()
inventory_df = tr.t_inventory_df()
page_views_mod = tr.t_page_views_mod()
envios_df_mod = tr.t_envios_medios_propios()
app_click_start_apartado = tr.t_app_on_click_start_apartado()
unique_visitors_pdp = tr.t_unique_visitors_pdp()
registros_funnel_vendedor = tr.t_registros_funnel_vendedor()
business_case_by_date = tr.t_business_case_by_date(business_case_df)
multiapartados_by_date = tr.t_multiapartados_by_date()
publicaciones_espacio = tr.t_publicaciones_por_espacio()
cancelaciones_date = tr.t_cancelaciones_by_date()
plp_date = tr.t_unique_visitors_plp()
#
clientes_apartado_efectivo = tr.t_clientes_con_apartado_month()
